## Введение

Часто бывает полезно разбивать объекты на категории не по количеству, а по качеству. Эта качественная информация нередко представляется как принадлежность наблюдения к отдельной категории, такой как пол, цвет, или марка автомобиля. Однаков не все категориальные данные одинаковые. Наборы категорий без внутреннего упорядочения называются номинальными. 
Пример номинальных категорий:
 * Синий, красный, зеленый
 * мужчина, женщина
 * банан, клубника, яблоко
 С другой стороны, когда набор категорий имеет некое естественное упорядочение, его называют порядковым. Например:
 * низкий, средний, высокий
 * молодые, старые
 * согласен, нейтрален, не согласен
 Более того, категориальная информация часто представлена в данных в виду вектора или столбца символьных значений (например, "Мэн", "Техас", "Делавер"). Проблема в том, что большинство машинно-обучающихся алгоритмов требуют ввода числовых значений.
 
 Алгоритм k ближайших соседей предоставляет простой пример. Одним из шагов в алгоритме является вычисление расстояния между наблюдениями - часто с использованием евклидова расстояния:
 
 $\sqrt{\sum_{i=1}^{n} (x_i-y_i)^2}$
 
 где x и y - это два наблюдения; i - номер признака наблюдений.
 Однако вычисление расстояния, очевидно, невозможно, если значение $x_i$ является строковым типом (например, "Техас"). Для того чтобы его можно было ввести в уравнение евклидова расстояния, нужно преобразовать строковое значение в числовой формат. Наша цель - сделать преобразование, которое правильно передает информацию в категориях (упорядоченность, относительные интервалы между категориями и тд)

##  Кодирование номинальных категориальных признаков

Дан признак с номинальными классами, который не имеет внутренней упорядоченности (например, яблоко, груша, банан)

In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

feature = np.array([['Texas'],
                   ['California'],
                   ['Delaware'],
                   ['Texas'],
                   ['Moscow'],
                   ['Kotlas']])
# Создать кодировщик одного активного состояния
one_hot = LabelBinarizer()

# Преобразовать признак в кодировку с одним активным состоянием
one_hot.fit_transform(feature)

array([[0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]])

In [2]:
one_hot.classes_

array(['California', 'Delaware', 'Kotlas', 'Moscow', 'Texas'],
      dtype='<U10')

In [3]:
# Обрабить кодирование
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Delaware', 'Texas', 'Moscow', 'Kotlas'],
      dtype='<U10')

In [4]:
# Аналог в pandas
import pandas as pd

pd.get_dummies(feature[:,0])

,California,Delaware,Kotlas,Moscow,Texas
0,0,0,0,0,1
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,0,0,1
4,0,0,0,1,0
5,0,0,1,0,0


In [5]:
# Для работы с мудьтиклассовыми признаками
multiclass_feature = [('Texas', 'Florida'),
                     ('California', 'Alabama'),
                     ('Texas', 'Florida'),
                     ('Delawer','Florida'),
                     ('Texas','Alabama')]

# Создать мультиклассовый кодировщик, преобразующий признак в кодировку с одним к=активным состоянием
one_hot_multiclass = MultiLabelBinarizer()

# Кодировать мультиклассовый признак в кодировку с одним активным состоянием
one_hot_multiclass.fit(multiclass_feature)

MultiLabelBinarizer()

In [6]:
one_hot_multiclass.transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [8]:
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delawer', 'Florida', 'Texas'],
      dtype=object)

## Кодирование порядковых категориальных признаков

Выполнить кодировку порядкового категориального признак (высокий, средний, низкий)


In [9]:
# Использовать метод replace фрейма данных pandas для преобразование строковых меток в числовые эквиваленты

import pandas as pd 
dataframe = pd.DataFrame({"оценка": ["низкая","низкая","средняя","средняя","высокая"]})

# Создать словарь преобразования шкалы
scale_mapper = {"низкая":1,
               "средняя":2,
               "высокая":3}
# заменить значения признаков значениями словаря
dataframe["оценка"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: оценка, dtype: int64

## Кодирование словарей признаков

Дан словарь, и требуется его конвертировать в матрицу признаков

In [16]:
# Используется класс-векторизатор словаря DictVectorizer

from sklearn.feature_extraction import DictVectorizer

data_dict = [{"красный": 2, "синий":4},
            {"красный": 4, "синий":3},
            {"красный": 1, "желтый":2},
            {"красный": 2, "желтый":2}]

dictvectorizer = DictVectorizer(sparse = False)
dictvectorizer

DictVectorizer(sparse=False)

In [17]:
features = dictvectorizer.fit_transform(data_dict)

features

array([[0., 2., 4.],
       [0., 4., 3.],
       [2., 1., 0.],
       [2., 2., 0.]])

In [18]:
feature_names = dictvectorizer.get_feature_names_out()

feature_names

array(['желтый', 'красный', 'синий'], dtype=object)

In [20]:
import pandas as pd

pd.DataFrame(features, columns = feature_names)

,желтый,красный,синий
0,0.0,2.0,4.0
1,0.0,4.0,3.0
2,2.0,1.0,0.0
3,2.0,2.0,0.0


##  Импутация пропущенных значений классов

In [31]:
# Дан категориальный признак содержащий пропущенные значения, которые требуется заменить предсказанными значениями

import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# Создать матрицу категориальных признаков 
X = np.array([[0,2.10,1.45],
             [1, 1.18, 1.33],
             [0,1.22,1.27],
             [1,-0.21,-1.19]])
# Создать матрицу признаков с отсутсвующими значениями
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])
# Натренировать ученика KNN
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:],X[:,0])

# Предсказать класс пропущенных значений
imputed_values = trained_model.predict(X_with_nan[:,1:])

In [26]:
X_with_nan[:,1:]

array([[ 0.87,  1.31],
       [-0.67, -0.22]])

In [27]:
imputed_values

array([0., 1.])

In [32]:
# Соединить столбец предсказанного класса с другими признаками
X_with_imputed = np.hstack((imputed_values.reshape(-1,1),X_with_nan[:,1:]))

np.vstack((X_with_imputed,X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [36]:
# Альтернативным решением является заполнение пропущенных значений наиболее частыми значениями признаков

from sklearn.impute import SimpleImputer

# Соединить две матрицы признаков
X_complete = np.vstack((X_with_nan, X))
imputer = SimpleImputer(strategy = 'most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

## Работа с несбалансированными классами

Дан вектор целей с очень несбалансированными классами

Решение
* Собрать больше данных.
* Изменить метрические показатели используемые для оценивания модели
* Рассмотреть возможность использования встроенных в модель параметров веса классов, делая понижающий или повышающий отбор


Для того чтобы продемонстрировать наши решения, нам нужно создать немного
данных с несбалансированными классами. Набор данных ирисов Фишера содержит
три сбалансированных класса по 50 наблюдений, каждый из которых указывает на
ирис щетинистый (Iris setosa), ирис виргинский (Iris virginica) и ирис
разноцветный (Iris versicolor). Для того чтобы разбалансировать набор данных, мы
удаляем 40 из 50 наблюдений ириса щетинистого, а затем объединяем классы ирис
виргинский и ирис разноцветный. Конечным результатом является бинарный век-
тор целей, указывающий на то, является ли наблюдение цветком ириса щетини-
стого или нет. Результатом станут 10 наблюдений ирис щетинистый (класс 0) и
100 наблюдений не ирис щетинистый (класс 1):

In [47]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()

features = iris.data

target = iris.target

features = features[40:,:]
target = target[40:]

# Создать бинарный вектор целей, указывающий, является ли класс 0
target = np.where((target == 0), 0, 1)

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [51]:
# Создать веса
weights = {0: .9, 1: .1}

# Создаем классификатор на основе случайного леса с весами
RandomForestClassifier(class_weight=weights)
RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=.0, min_samples_split=2, min_samples_leaf=1
                       , min_weight_fraction_leaf=.0, n_estimators=10, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0,warm_start = False)

RandomForestClassifier(max_features='auto', n_estimators=10, n_jobs=1)

In [59]:
# Индексы наблюдений каждого класса
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# Количество наблюдений в каждом классе
n_class0 = len(i_class0)
n_class1 = len(i_class1)

#  Для каждого наблюдения класса 0 сделать случайную выборку
# Из класса 1 без возврата
i_class1_downsampled = np.random.choice(i_class1, size = n_class0, replace=False)

# Соединить вектор целей класса 0 с вектором целей пониженного класса 1
np.hstack((target[i_class0], target[i_class1_downsampled]))


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [61]:
np.vstack((features[i_class0,:], features[i_class1_downsampled, :]))[0:5]


array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

In [65]:
# Для каждого наблюдения в классе 1 сделать выборку из класса 0 с возвратом
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace = True)

# Соединить повышенный вектор целей класса 0 с вектором целей класса 1
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [66]:
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

array([[5. , 3.5, 1.6, 0.6],
       [4.5, 2.3, 1.3, 0.3],
       [4.6, 3.2, 1.4, 0.2],
       [4.8, 3. , 1.4, 0.3],
       [4.8, 3. , 1.4, 0.3]])